# Gain Statistical Insights into Your DataTable

Woodwork provides two methods on DataTable to allow users to better understand their data: `describe` and `get_mutual_information`. The added typing information in a DataTable allows us to more specifically choose which operations to apply to each column according to what's possible for each type.

First, we'll build a DataTable with a variety of types so that we can see the full capabilityies of `describe` and `get_mutual_information`. We'll set an index as well as define a few Logical Types.

In [ ]:
import pandas as pd
from woodwork import DataTable

df = pd.DataFrame({
        'id': range(3),
        'country': ['AUS', 'GB', 'NZ'],
        'email': ['john.smith@example.com', None, 'team@featuretools.com'],
        'delta': (pd.Series([pd.to_datetime('2020-09-01')] * 3) - pd.to_datetime('2020-07-01')),
        'age': [33, 25, 31],
        'signup_date': [pd.to_datetime('2020-09-01')] * 3,
        'is_registered': [True, False, True],
    })

dt = DataTable(df, index='id')
dt = dt.set_logical_types({'email':'EmailAddress', 'country': 'CountryCode'})

In order to understand which Logical Types `describe` and `get_mutual_information` will include in their calculations, we can split the available Logical Types into five categories:

- Categorical
    - `Categorical`, `CountryCode`, `Ordinal`, `SubRegionCode`, `ZIPCode`
- Numeric
    - `Double`, `Integer`, `WholeNumber`
- String
    - `EmailAddress`, `FilePath`, `FullName`, `IPAddress`, `LatLong`, `NaturalLanguage`, `PhoneNumber`, `URL`
- Boolean - just the `Boolean` LogicalType
- Datetime - just the `Datetime` LogicalType
- Timedelta - just the `Timedelta` LogicalType


## DataTable.describe

Using `dt.describe()` will calculate statistics for the columns in your DataTable and return a Pandas DataFrame with the relevant calculations done for each column.

The statistics calculated can be broken down into a few types:

- General - can be applied to all columns
    - `nan_count` and `mode`
- Aggregate
    - `count` - Categorical, Numeric, Datetime
    - `nunique` - Categorical, Numeric, Datetime
    - `mean` - Numeric, Datetime
    - `std` - Numeric
    - `min` - Numeric, Datetime
    - `max` - Numeric, Datetime
- Boolean - only relevant for columns of Booleans
    - `num_false` and `num_true`
- Quartile - calculated on Numeric columns
    - `first_quartile`
    - `second_quartile`
    - `third_quartile`

Now, let's call `describe` and see how the above statistics get calculated.

In [ ]:
dt.describe()

There are a couple things to note in the above dataframe:

- The DataTable's `index` is not included
- We provide typing information for each column according to Woodwork's typing system
- Any statistic that cannot be calculated for a column will be filled with `NaN`.
- The `email` column contains a null value, which does not get counted in any of the calculations other than `nunique`. 

## DataTable.get_mutual_information()

`dt.get_mutual_information` will calculate the mutual information between all pairs of columns whose Logical Types fall into one of the Numeric, Categorical, or Boolean categories described above. The mutual information between columns `A` and `B` can be understood as the amount of knowlege we can have about column `A` if we have the values of column `B`. The more mutual information there is between `A` and `B`, the less uncertainty there is in `A` knowing `B` or vice versa. 

The full list of Logical Types for which mutual information can be calculated is Boolean, Categorical, CountryCode, Double, Integer, Ordinal, SubRegionCode, WholeNumber, and ZIPCode.

If we call `dt.get_mutual_information()`, we'll see that `delta`, `signup_date`, and `email` will be excluded from the resulting dataframe.

In [ ]:
dt.get_mutual_information()

We see that the mutual information between a column and itself is 1, which makes sense since if we know everything about a column given itself.

#### Available Parameters
`get_mutual_information` provides two parameters for tuning the mutual information calculation.

- `num_bins` - In order to calculate mutual information on continuos data, we bin numeric data into categories. `num_bins` allows users to choose the number of bins with which to categorize data.
    - Defaults to using 10 bins/
    - The more bins there are, the more variety a column will have and, theremore, the lower the mutual information will be. The number of bins used shoudl accurately portray the spread of the data.
- `nrows` - if an `nrows` is set at a value below the number of rows in the DataTable, `nrows` will be randomly sampled from the underlying data
    - Defaults to using all the available rows.
    - Decreasing the number of rows can speed up the mutual information calculation on a DataTable with many rows, though care should be taken that the number being sampled is large enough to accurately portray the data.

While the above dataframe nicely shows how `get_mutual_information` includes only some types of columns, it doesn't actually provide much in terms of understanding mutual information because our dummy DataTable only has 3 rows of data.

If we use our demo retail DataTable, we can see the behavior of mutual information more clearly.


## Retail Demo

The retail demo DataTable can be used to look at a larger sample dataset.

Let's explore the dataset. First we'll just take a look at the type information and the underlying data.

In [ ]:
from woodwork.demo import load_retail

dt = load_retail()
dt.types

We can understand this dataset to contain information on retail shopping. We can see that there is an index set at `order_product_id` and a time index at `order_date`. 

Now, we're going to get an overview of the data using `describe`.

In [ ]:
dt.describe()

We can see that our data has more than 400K rows and no null values among other statistics calculated.

If we want to know about how the columns might relate to one antoher, we can take a look at the mutual information for the dataset.

In [ ]:
mi = dt.get_mutual_information()
mi[mi['column_1'] != mi['column_2']].sort_values('mutual_info', ascending=False).head(10)

Let's see what happens if we decrease the number of rows that we pull for the DataTable down to 50,000:

In [ ]:
mi = dt.get_mutual_information(nrows=20000)
mi[mi['column_1'] != mi['column_2']].sort_values('mutual_info', ascending=False).head(10)

Between 400K and 50K rows, we do see some differences in the mutual information calculated, but many pairs of columns are present in both runs. However, we will see consistensy across 50K runs even though the sampled rows will be different.

In [ ]:
mi = dt.get_mutual_information(nrows=20000)
mi[mi['column_1'] != mi['column_2']].sort_values('mutual_info', ascending=False).head(10)

Now we'll explore changing the number of bins. Note that this will only impact numeric columns `quantity` and `unit_price`. We're going to increase the number of bins from 10 to 50, only showing the impacted columns.

In [ ]:
mi = dt.get_mutual_information()
p = mi[mi['column_1'] != mi['column_2']].sort_values('mutual_info', ascending=False)
p[p['column_1'].isin(['unit_price', 'quantity']) | p['column_2'].isin(['unit_price', 'quantity'])]

In [ ]:
mi = dt.get_mutual_information(num_bins = 50)
p = mi[mi['column_1'] != mi['column_2']].sort_values('mutual_info', ascending=False)
p[p['column_1'].isin(['unit_price', 'quantity']) | p['column_2'].isin(['unit_price', 'quantity'])]